In [3]:
import pandas as pd
import os
import librosa
import librosa.display
import matplotlib.pyplot as plt
from sklearn.preprocessing import normalize
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pickle
import joblib
from sklearn.model_selection import train_test_split
from tensorflow.keras import models, layers
import tensorflow as tf

Class and Class ID :-
1. Asthma
2. Bronchiectasis
3. Bronchiolitis
4. Copd
5. Healthy
6. LRTI
7. Pneumonia
8. URTI

In [4]:
df=pd.read_csv('Meta_Data_of_audio_files.csv')
df=df.drop(0)
display(df)
df_feat=df
file=df_feat['File_Path']
classid=df_feat['Class_ID']
df_feat

,Unnamed: 0,File_Path,Class_ID,Class
1,1,Audio files/Copd/104_1b1_Ar_sc_Litt3200.wav,4,Copd
2,2,Audio files/Copd/104_1b1_Ll_sc_Litt3200.wav,4,Copd
3,3,Audio files/Copd/104_1b1_Lr_sc_Litt3200.wav,4,Copd
4,4,Audio files/Copd/104_1b1_Pl_sc_Litt3200.wav,4,Copd
5,5,Audio files/Copd/104_1b1_Pr_sc_Litt3200.wav,4,Copd
...,...,...,...,...
883,883,Audio files/URTI/190_1b1_Tc_sc_Meditron.wav,8,URTI
884,884,Audio files/URTI/197_1b1_Al_sc_Meditron.wav,8,URTI
885,885,Audio files/URTI/197_1b1_Tc_sc_Meditron.wav,8,URTI
886,886,Audio files/URTI/210_1b1_Al_sc_Meditron.wav,8,URTI


,Unnamed: 0,File_Path,Class_ID,Class
1,1,Audio files/Copd/104_1b1_Ar_sc_Litt3200.wav,4,Copd
2,2,Audio files/Copd/104_1b1_Ll_sc_Litt3200.wav,4,Copd
3,3,Audio files/Copd/104_1b1_Lr_sc_Litt3200.wav,4,Copd
4,4,Audio files/Copd/104_1b1_Pl_sc_Litt3200.wav,4,Copd
5,5,Audio files/Copd/104_1b1_Pr_sc_Litt3200.wav,4,Copd
...,...,...,...,...
883,883,Audio files/URTI/190_1b1_Tc_sc_Meditron.wav,8,URTI
884,884,Audio files/URTI/197_1b1_Al_sc_Meditron.wav,8,URTI
885,885,Audio files/URTI/197_1b1_Tc_sc_Meditron.wav,8,URTI
886,886,Audio files/URTI/210_1b1_Al_sc_Meditron.wav,8,URTI


In [24]:
df.Class_ID.unique()

array([4, 6, 7, 8], dtype=int64)

In [5]:
#for padding data
def padding(array, xx, yy):
    """
    :param array: numpy array
    :param xx: desired height
    :param yy: desired width
    :return: padded array
    """
    h = array.shape[0]
    w = array.shape[1]
    a = max((xx - h) // 2,0)
    aa = max(0,xx - a - h)
    b = max(0,(yy - w) // 2)
    bb = max(yy - b - w,0)
    return np.pad(array, pad_width=((a, aa), (b, bb)), mode='constant')

In [9]:
# def generate_features(y_cut):
#     sr=28000
#     max_size=4800 #my max audio file feature width
#     stft = padding(np.abs(librosa.stft(y_cut, n_fft=255, hop_length = 512)), 128, max_size)
#     MFCCs = padding(librosa.feature.mfcc(y_cut, n_fft=255, hop_length=512,n_mfcc=128),128,max_size)
#     spec_centroid = librosa.feature.spectral_centroid(y=y_cut, sr=sr)
#     chroma_stft = librosa.feature.chroma_stft(y=y_cut, sr=sr)
#     spec_bw = librosa.feature.spectral_bandwidth(y=y_cut, sr=sr)
#     #Now the padding part
#     image = np.array([padding(normalize(spec_bw),1, max_size)]).reshape(1,max_size)
#     image = np.append(image,padding(normalize(spec_centroid),1, max_size), axis=0) 
#     #repeat the padded spec_bw,spec_centroid and chroma stft until they are stft and MFCC-sized
#     for i in range(0,9):
#         image = np.append(image,padding(normalize(spec_bw),1, max_size), axis=0)
#         image = np.append(image, padding(normalize(spec_centroid),1, max_size), axis=0)
#         image = np.append(image, padding(normalize(chroma_stft),12, max_size), axis=0)
#     image=np.dstack((image,np.abs(stft)))
#     image=np.dstack((image,MFCCs))
#     return image

<b> Only STFT and MFCC fusion </b>

In [6]:
def generate_features(y_cut):
    sr=28000
    max_size=50 #my max audio file feature width
    stft = padding(np.abs(librosa.stft(y_cut, n_fft=255, hop_length = 512)), 128, max_size)
    MFCCs = padding(librosa.feature.mfcc(y_cut, n_fft=255, hop_length=512,n_mfcc=128),128,max_size)
    image=np.array(np.abs(stft))
    image=np.dstack((image,MFCCs))
    return image

Cutting 5, 7, 8 into multiple clips

In [28]:
def get_features(df_in):   
    features=[]     
    labels = [] #empty array to store labels     
    #For each species, determine how many augmentations are needed
    df_in=df_in.reset_index()     
    for i in df_in.Class_ID.unique():
           print('Class_ID:',i)    
           #all the file indices with the same species_id     
           filelist = df_in.loc[df_in.Class_ID == i].index 
           #filelist = df_in.loc[df_in.Class_ID == 3].index
           for j in range(0,len(filelist)):             
                        filename = df_in.iloc[filelist[j]].File_Path #get the filename   
                        #define the beginning time of the signal          
                        #tstart = df_in.iloc[filelist[j]].t_min             
                        #tend = df_in.iloc[filelist[j]].t_max #end of signal
                        #recording_id = df_in.iloc[filelist[j]].recording_id
                        Class_ID = i
                        #songtype_id = df_in.iloc[filelist[j]].songtype_id   
                        #Load the file
                        ys, sr = librosa.load(filename,sr=28000)  
                        print(filename + ' '+ 'going on')
                        print(str(j+1)+' out of '+ str(len(filelist))+'\n')
                        #cut the file to signal start and end  
                        n=0
                        if(i==4):
                            ys=ys[int(round(0*sr)):int(round(6.8*sr))]  
                            #generate features & output numpy array          
                            data = generate_features(ys) 
                            print(data.shape)
                            features.append(data[np.newaxis,...])    
                            labels.append(Class_ID)  
                        else:
                            shape=400
                            while(shape>372):
                                ys=ys[int(round(n*sr)):int(round(6.8*(n+1)*sr))]  
                                #generate features & output numpy array          
                                data = generate_features(ys) 
                                shape
                                print(data.shape)
                                features.append(data[np.newaxis,...])    
                                labels.append(Class_ID)  
                                shape=data.shape[1]
                                n=n+1
    output=np.concatenate(features,axis=0)     
    return(np.array(output), labels)

In [10]:
# def get_features(df_in):   
#     features=[]     
#     labels = [] #empty array to store labels     
#     #For each species, determine how many augmentations are needed
#     df_in=df_in.reset_index()     
#     for i in [8]:#df_in.Class_ID.unique():
#            print('Class_ID:',i)    
#            #all the file indices with the same species_id     
#            filelist = df_in.loc[df_in.Class_ID == i].index         
#            #filelist = df_in.loc[df_in.Class_ID == 3].index
#            for j in range(0,len(filelist)):             
#                         filename = df_in.iloc[filelist[j]].File_Path #get the filename   
#                         #define the beginning time of the signal          
#                         #tstart = df_in.iloc[filelist[j]].t_min             
#                         #tend = df_in.iloc[filelist[j]].t_max #end of signal
#                         #recording_id = df_in.iloc[filelist[j]].recording_id
#                         Class_ID = i
#                         #songtype_id = df_in.iloc[filelist[j]].songtype_id   
#                         #Load the file
#                         ys, sr = librosa.load(filename,sr=28000)  
#                         print(filename + ' '+ 'going on')
#                         print(str(j)+' out of '+ str(len(filelist))+'\n')
#                         #cut the file to signal start and end  
#                         #y_cut=y[int(round(tstart*sr)):int(round(tend*sr))]  
#                         #generate features & output numpy array          
#                         data = generate_features(ys) 
#                         features.append(data[np.newaxis,...])    
#                         labels.append(Class_ID)     
#     X=np.concatenate(features,axis=0)     
#     y=labels
#     return 0

In [7]:
X, y=get_features(pd.concat([file, classid], axis=1))

Class_ID: 4
Audio files/Copd/104_1b1_Ar_sc_Litt3200.wav going on
0 out of 792

Audio files/Copd/104_1b1_Ll_sc_Litt3200.wav going on
1 out of 792

Audio files/Copd/104_1b1_Lr_sc_Litt3200.wav going on
2 out of 792

Audio files/Copd/104_1b1_Pl_sc_Litt3200.wav going on
3 out of 792

Audio files/Copd/104_1b1_Pr_sc_Litt3200.wav going on
4 out of 792

Audio files/Copd/106_2b1_Pl_mc_LittC2SE.wav going on
5 out of 792

Audio files/Copd/106_2b1_Pr_mc_LittC2SE.wav going on
6 out of 792

Audio files/Copd/107_2b3_Al_mc_AKGC417L.wav going on
7 out of 792

Audio files/Copd/107_2b3_Ar_mc_AKGC417L.wav going on
8 out of 792

Audio files/Copd/107_2b3_Ll_mc_AKGC417L.wav going on
9 out of 792

Audio files/Copd/107_2b3_Lr_mc_AKGC417L.wav going on
10 out of 792

Audio files/Copd/107_2b3_Pl_mc_AKGC417L.wav going on
11 out of 792

Audio files/Copd/107_2b3_Pr_mc_AKGC417L.wav going on
12 out of 792

Audio files/Copd/107_2b3_Tc_mc_AKGC417L.wav going on
13 out of 792

Audio files/Copd/107_2b4_Al_mc_AKGC417L.wav go

Audio files/Copd/130_2b4_Pl_mc_AKGC417L.wav going on
121 out of 792

Audio files/Copd/130_2p3_Pl_mc_AKGC417L.wav going on
122 out of 792

Audio files/Copd/130_2p5_Al_mc_AKGC417L.wav going on
123 out of 792

Audio files/Copd/130_2p5_Ar_mc_AKGC417L.wav going on
124 out of 792

Audio files/Copd/130_2p5_Lr_mc_AKGC417L.wav going on
125 out of 792

Audio files/Copd/130_2p5_Pl_mc_AKGC417L.wav going on
126 out of 792

Audio files/Copd/130_2p5_Pr_mc_AKGC417L.wav going on
127 out of 792

Audio files/Copd/130_2p5_Tc_mc_AKGC417L.wav going on
128 out of 792

Audio files/Copd/130_3b3_Ll_mc_AKGC417L.wav going on
129 out of 792

Audio files/Copd/130_3b4_Al_mc_AKGC417L.wav going on
130 out of 792

Audio files/Copd/130_3b4_Ar_mc_AKGC417L.wav going on
131 out of 792

Audio files/Copd/130_3b4_Lr_mc_AKGC417L.wav going on
132 out of 792

Audio files/Copd/130_3b4_Pl_mc_AKGC417L.wav going on
133 out of 792

Audio files/Copd/130_3b4_Pr_mc_AKGC417L.wav going on
134 out of 792

Audio files/Copd/130_3p2_Al_mc_AKG

Audio files/Copd/147_2b3_Al_mc_AKGC417L.wav going on
240 out of 792

Audio files/Copd/147_2b3_Ar_mc_AKGC417L.wav going on
241 out of 792

Audio files/Copd/147_2b3_Ll_mc_AKGC417L.wav going on
242 out of 792

Audio files/Copd/147_2b3_Lr_mc_AKGC417L.wav going on
243 out of 792

Audio files/Copd/147_2b3_Pl_mc_AKGC417L.wav going on
244 out of 792

Audio files/Copd/147_2b4_Al_mc_AKGC417L.wav going on
245 out of 792

Audio files/Copd/147_2b4_Ar_mc_AKGC417L.wav going on
246 out of 792

Audio files/Copd/147_2b4_Ll_mc_AKGC417L.wav going on
247 out of 792

Audio files/Copd/147_2b4_Lr_mc_AKGC417L.wav going on
248 out of 792

Audio files/Copd/147_2b4_Pl_mc_AKGC417L.wav going on
249 out of 792

Audio files/Copd/151_2p2_Al_mc_AKGC417L.wav going on
250 out of 792

Audio files/Copd/151_2p2_Ar_mc_AKGC417L.wav going on
251 out of 792

Audio files/Copd/151_2p2_Ll_mc_AKGC417L.wav going on
252 out of 792

Audio files/Copd/151_2p2_Lr_mc_AKGC417L.wav going on
253 out of 792

Audio files/Copd/151_2p2_Pl_mc_AKG

Audio files/Copd/160_1b3_Tc_mc_AKGC417L.wav going on
359 out of 792

Audio files/Copd/160_1b4_Al_mc_AKGC417L.wav going on
360 out of 792

Audio files/Copd/160_1b4_Ar_mc_AKGC417L.wav going on
361 out of 792

Audio files/Copd/160_1b4_Lr_mc_AKGC417L.wav going on
362 out of 792

Audio files/Copd/160_1b4_Pl_mc_AKGC417L.wav going on
363 out of 792

Audio files/Copd/160_1b4_Pr_mc_AKGC417L.wav going on
364 out of 792

Audio files/Copd/160_1b4_Tc_mc_AKGC417L.wav going on
365 out of 792

Audio files/Copd/160_2b3_Lr_mc_AKGC417L.wav going on
366 out of 792

Audio files/Copd/160_2b4_Ar_mc_AKGC417L.wav going on
367 out of 792

Audio files/Copd/160_2b4_Pl_mc_AKGC417L.wav going on
368 out of 792

Audio files/Copd/160_2b4_Pr_mc_AKGC417L.wav going on
369 out of 792

Audio files/Copd/160_2b4_Tc_mc_AKGC417L.wav going on
370 out of 792

Audio files/Copd/162_1b2_Al_mc_AKGC417L.wav going on
371 out of 792

Audio files/Copd/162_1b2_Ar_mc_AKGC417L.wav going on
372 out of 792

Audio files/Copd/162_1b2_Ll_mc_AKG

Audio files/Copd/174_1p4_Ll_mc_AKGC417L.wav going on
478 out of 792

Audio files/Copd/174_1p4_Lr_mc_AKGC417L.wav going on
479 out of 792

Audio files/Copd/174_1p4_Pl_mc_AKGC417L.wav going on
480 out of 792

Audio files/Copd/174_1p4_Pr_mc_AKGC417L.wav going on
481 out of 792

Audio files/Copd/174_1p4_Tc_mc_AKGC417L.wav going on
482 out of 792

Audio files/Copd/174_2p3_Al_mc_AKGC417L.wav going on
483 out of 792

Audio files/Copd/174_2p3_Ar_mc_AKGC417L.wav going on
484 out of 792

Audio files/Copd/174_2p3_Pl_mc_AKGC417L.wav going on
485 out of 792

Audio files/Copd/174_2p3_Pr_mc_AKGC417L.wav going on
486 out of 792

Audio files/Copd/174_2p3_Tc_mc_AKGC417L.wav going on
487 out of 792

Audio files/Copd/175_1b1_Al_sc_Litt3200.wav going on
488 out of 792

Audio files/Copd/175_1b1_Ar_sc_Litt3200.wav going on
489 out of 792

Audio files/Copd/175_1b1_Ll_sc_Litt3200.wav going on
490 out of 792

Audio files/Copd/175_1b1_Lr_sc_Litt3200.wav going on
491 out of 792

Audio files/Copd/175_1b1_Pl_sc_Lit

Audio files/Copd/192_2b1_Ar_mc_LittC2SE.wav going on
597 out of 792

Audio files/Copd/192_2b2_Al_mc_LittC2SE.wav going on
598 out of 792

Audio files/Copd/192_2b2_Ar_mc_LittC2SE.wav going on
599 out of 792

Audio files/Copd/192_2b3_Al_mc_LittC2SE.wav going on
600 out of 792

Audio files/Copd/192_2b3_Ar_mc_LittC2SE.wav going on
601 out of 792

Audio files/Copd/193_1b2_Al_mc_AKGC417L.wav going on
602 out of 792

Audio files/Copd/193_1b2_Ar_mc_AKGC417L.wav going on
603 out of 792

Audio files/Copd/193_1b2_Ll_mc_AKGC417L.wav going on
604 out of 792

Audio files/Copd/193_1b2_Pl_mc_AKGC417L.wav going on
605 out of 792

Audio files/Copd/193_1b2_Pr_mc_AKGC417L.wav going on
606 out of 792

Audio files/Copd/193_1b2_Tc_mc_AKGC417L.wav going on
607 out of 792

Audio files/Copd/193_1b4_Lr_mc_AKGC417L.wav going on
608 out of 792

Audio files/Copd/193_7b3_Al_mc_AKGC417L.wav going on
609 out of 792

Audio files/Copd/193_7b3_Ar_mc_AKGC417L.wav going on
610 out of 792

Audio files/Copd/193_7b3_Ll_mc_AKG

Audio files/Copd/207_2b3_Al_mc_AKGC417L.wav going on
716 out of 792

Audio files/Copd/207_2b3_Ar_mc_AKGC417L.wav going on
717 out of 792

Audio files/Copd/207_2b3_Pl_mc_AKGC417L.wav going on
718 out of 792

Audio files/Copd/207_2b3_Pr_mc_AKGC417L.wav going on
719 out of 792

Audio files/Copd/207_2b3_Tc_mc_AKGC417L.wav going on
720 out of 792

Audio files/Copd/207_2b4_Al_mc_AKGC417L.wav going on
721 out of 792

Audio files/Copd/207_2b4_Ar_mc_AKGC417L.wav going on
722 out of 792

Audio files/Copd/207_2b4_Pl_mc_AKGC417L.wav going on
723 out of 792

Audio files/Copd/207_2b4_Pr_mc_AKGC417L.wav going on
724 out of 792

Audio files/Copd/207_2b4_Tc_mc_AKGC417L.wav going on
725 out of 792

Audio files/Copd/207_3b2_Al_mc_AKGC417L.wav going on
726 out of 792

Audio files/Copd/207_3b2_Ar_mc_AKGC417L.wav going on
727 out of 792

Audio files/Copd/207_3b2_Lr_mc_AKGC417L.wav going on
728 out of 792

Audio files/Copd/207_3b2_Pl_mc_AKGC417L.wav going on
729 out of 792

Audio files/Copd/207_3b2_Pr_mc_AKG

Audio files/Pneumonia/122_2b3_Ar_mc_LittC2SE.wav going on
7 out of 37

Audio files/Pneumonia/122_2b3_Tc_mc_LittC2SE.wav going on
8 out of 37

Audio files/Pneumonia/135_2b1_Al_mc_LittC2SE.wav going on
9 out of 37

Audio files/Pneumonia/135_2b1_Ar_mc_LittC2SE.wav going on
10 out of 37

Audio files/Pneumonia/135_2b1_Pl_mc_LittC2SE.wav going on
11 out of 37

Audio files/Pneumonia/135_2b1_Tc_mc_LittC2SE.wav going on
12 out of 37

Audio files/Pneumonia/135_2b2_Al_mc_LittC2SE.wav going on
13 out of 37

Audio files/Pneumonia/135_2b2_Ar_mc_LittC2SE.wav going on
14 out of 37

Audio files/Pneumonia/135_2b2_Pl_mc_LittC2SE.wav going on
15 out of 37

Audio files/Pneumonia/135_2b2_Tc_mc_LittC2SE.wav going on
16 out of 37

Audio files/Pneumonia/135_2b3_Al_mc_LittC2SE.wav going on
17 out of 37

Audio files/Pneumonia/135_2b3_Ar_mc_LittC2SE.wav going on
18 out of 37

Audio files/Pneumonia/135_2b3_Pl_mc_LittC2SE.wav going on
19 out of 37

Audio files/Pneumonia/135_2b3_Pr_mc_LittC2SE.wav going on
20 out of

In [29]:
X7cut, y7cut=get_features(pd.concat([file, classid], axis=1))

Class_ID: 4
Audio files/Copd/104_1b1_Ar_sc_Litt3200.wav going on
1 out of 792

(128, 372, 2)
Audio files/Copd/104_1b1_Ll_sc_Litt3200.wav going on
2 out of 792

(128, 372, 2)
Audio files/Copd/104_1b1_Lr_sc_Litt3200.wav going on
3 out of 792

(128, 372, 2)
Audio files/Copd/104_1b1_Pl_sc_Litt3200.wav going on
4 out of 792

(128, 372, 2)
Audio files/Copd/104_1b1_Pr_sc_Litt3200.wav going on
5 out of 792

(128, 372, 2)
Audio files/Copd/106_2b1_Pl_mc_LittC2SE.wav going on
6 out of 792

(128, 372, 2)
Audio files/Copd/106_2b1_Pr_mc_LittC2SE.wav going on
7 out of 792

(128, 372, 2)
Audio files/Copd/107_2b3_Al_mc_AKGC417L.wav going on
8 out of 792

(128, 372, 2)
Audio files/Copd/107_2b3_Ar_mc_AKGC417L.wav going on
9 out of 792

(128, 372, 2)
Audio files/Copd/107_2b3_Ll_mc_AKGC417L.wav going on
10 out of 792

(128, 372, 2)
Audio files/Copd/107_2b3_Lr_mc_AKGC417L.wav going on
11 out of 792

(128, 372, 2)
Audio files/Copd/107_2b3_Pl_mc_AKGC417L.wav going on
12 out of 792

(128, 372, 2)
Audio files/C

Audio files/Copd/130_1p4_Pl_mc_AKGC417L.wav going on
101 out of 792

(128, 372, 2)
Audio files/Copd/130_1p4_Pr_mc_AKGC417L.wav going on
102 out of 792

(128, 372, 2)
Audio files/Copd/130_1p4_Tc_mc_AKGC417L.wav going on
103 out of 792

(128, 372, 2)
Audio files/Copd/130_2b2_Al_mc_AKGC417L.wav going on
104 out of 792

(128, 372, 2)
Audio files/Copd/130_2b2_Ar_mc_AKGC417L.wav going on
105 out of 792

(128, 372, 2)
Audio files/Copd/130_2b2_Ll_mc_AKGC417L.wav going on
106 out of 792

(128, 372, 2)
Audio files/Copd/130_2b2_Lr_mc_AKGC417L.wav going on
107 out of 792

(128, 372, 2)
Audio files/Copd/130_2b2_Pl_mc_AKGC417L.wav going on
108 out of 792

(128, 372, 2)
Audio files/Copd/130_2b2_Pr_mc_AKGC417L.wav going on
109 out of 792

(128, 372, 2)
Audio files/Copd/130_2b2_Tc_mc_AKGC417L.wav going on
110 out of 792

(128, 372, 2)
Audio files/Copd/130_2b3_Al_mc_AKGC417L.wav going on
111 out of 792

(128, 372, 2)
Audio files/Copd/130_2b3_Ar_mc_AKGC417L.wav going on
112 out of 792

(128, 372, 2)
Audi

Audio files/Copd/138_2p2_Pr_mc_AKGC417L.wav going on
200 out of 792

(128, 372, 2)
Audio files/Copd/138_2p2_Tc_mc_AKGC417L.wav going on
201 out of 792

(128, 372, 2)
Audio files/Copd/139_1b1_Al_sc_Litt3200.wav going on
202 out of 792

(128, 372, 2)
Audio files/Copd/139_1b1_Ar_sc_Litt3200.wav going on
203 out of 792

(128, 372, 2)
Audio files/Copd/139_1b1_Ll_sc_Litt3200.wav going on
204 out of 792

(128, 372, 2)
Audio files/Copd/139_1b1_Lr_sc_Litt3200.wav going on
205 out of 792

(128, 372, 2)
Audio files/Copd/139_1b1_Pl_sc_Litt3200.wav going on
206 out of 792

(128, 372, 2)
Audio files/Copd/139_1b1_Pr_sc_Litt3200.wav going on
207 out of 792

(128, 372, 2)
Audio files/Copd/141_1b1_Pr_mc_LittC2SE.wav going on
208 out of 792

(128, 372, 2)
Audio files/Copd/141_1b2_Ar_mc_LittC2SE.wav going on
209 out of 792

(128, 372, 2)
Audio files/Copd/141_1b2_Lr_mc_LittC2SE.wav going on
210 out of 792

(128, 372, 2)
Audio files/Copd/141_1b2_Pr_mc_LittC2SE.wav going on
211 out of 792

(128, 372, 2)
Audi

Audio files/Copd/154_4b4_Lr_mc_AKGC417L.wav going on
299 out of 792

(128, 372, 2)
Audio files/Copd/154_4b4_Pl_mc_AKGC417L.wav going on
300 out of 792

(128, 372, 2)
Audio files/Copd/154_4b4_Pr_mc_AKGC417L.wav going on
301 out of 792

(128, 372, 2)
Audio files/Copd/155_2b1_Al_mc_LittC2SE.wav going on
302 out of 792

(128, 372, 2)
Audio files/Copd/156_2b3_Al_mc_AKGC417L.wav going on
303 out of 792

(128, 372, 2)
Audio files/Copd/156_2b3_Ar_mc_AKGC417L.wav going on
304 out of 792

(128, 372, 2)
Audio files/Copd/156_2b3_Ll_mc_AKGC417L.wav going on
305 out of 792

(128, 372, 2)
Audio files/Copd/156_2b3_Lr_mc_AKGC417L.wav going on
306 out of 792

(128, 372, 2)
Audio files/Copd/156_2b3_Pl_mc_AKGC417L.wav going on
307 out of 792

(128, 372, 2)
Audio files/Copd/156_2b3_Pr_mc_AKGC417L.wav going on
308 out of 792

(128, 372, 2)
Audio files/Copd/156_5b3_Al_mc_AKGC417L.wav going on
309 out of 792

(128, 372, 2)
Audio files/Copd/156_5b3_Ar_mc_AKGC417L.wav going on
310 out of 792

(128, 372, 2)
Audi

Audio files/Copd/163_2b2_Ll_mc_AKGC417L.wav going on
398 out of 792

(128, 372, 2)
Audio files/Copd/163_2b2_Lr_mc_AKGC417L.wav going on
399 out of 792

(128, 372, 2)
Audio files/Copd/163_2b2_Pl_mc_AKGC417L.wav going on
400 out of 792

(128, 372, 2)
Audio files/Copd/163_2b2_Pr_mc_AKGC417L.wav going on
401 out of 792

(128, 372, 2)
Audio files/Copd/163_2b2_Tc_mc_AKGC417L.wav going on
402 out of 792

(128, 372, 2)
Audio files/Copd/163_8b3_Al_mc_AKGC417L.wav going on
403 out of 792

(128, 372, 2)
Audio files/Copd/163_8b3_Ar_mc_AKGC417L.wav going on
404 out of 792

(128, 372, 2)
Audio files/Copd/163_8b3_Ll_mc_AKGC417L.wav going on
405 out of 792

(128, 372, 2)
Audio files/Copd/163_8b3_Lr_mc_AKGC417L.wav going on
406 out of 792

(128, 372, 2)
Audio files/Copd/163_8b3_Pl_mc_AKGC417L.wav going on
407 out of 792

(128, 372, 2)
Audio files/Copd/163_8b3_Pr_mc_AKGC417L.wav going on
408 out of 792

(128, 372, 2)
Audio files/Copd/166_1p1_Al_sc_Meditron.wav going on
409 out of 792

(128, 372, 2)
Audi

Audio files/Copd/176_1b3_Ll_mc_AKGC417L.wav going on
497 out of 792

(128, 372, 2)
Audio files/Copd/176_1b3_Lr_mc_AKGC417L.wav going on
498 out of 792

(128, 372, 2)
Audio files/Copd/176_1b3_Pl_mc_AKGC417L.wav going on
499 out of 792

(128, 372, 2)
Audio files/Copd/176_1b3_Pr_mc_AKGC417L.wav going on
500 out of 792

(128, 372, 2)
Audio files/Copd/176_1b3_Tc_mc_AKGC417L.wav going on
501 out of 792

(128, 372, 2)
Audio files/Copd/176_1b4_Al_mc_AKGC417L.wav going on
502 out of 792

(128, 372, 2)
Audio files/Copd/176_1b4_Ar_mc_AKGC417L.wav going on
503 out of 792

(128, 372, 2)
Audio files/Copd/176_1b4_Ll_mc_AKGC417L.wav going on
504 out of 792

(128, 372, 2)
Audio files/Copd/176_1b4_Lr_mc_AKGC417L.wav going on
505 out of 792

(128, 372, 2)
Audio files/Copd/176_1b4_Pl_mc_AKGC417L.wav going on
506 out of 792

(128, 372, 2)
Audio files/Copd/176_1b4_Pr_mc_AKGC417L.wav going on
507 out of 792

(128, 372, 2)
Audio files/Copd/176_1b4_Tc_mc_AKGC417L.wav going on
508 out of 792

(128, 372, 2)
Audi

Audio files/Copd/189_1b2_Lr_mc_LittC2SE.wav going on
596 out of 792

(128, 372, 2)
Audio files/Copd/192_2b1_Al_mc_LittC2SE.wav going on
597 out of 792

(128, 372, 2)
Audio files/Copd/192_2b1_Ar_mc_LittC2SE.wav going on
598 out of 792

(128, 372, 2)
Audio files/Copd/192_2b2_Al_mc_LittC2SE.wav going on
599 out of 792

(128, 372, 2)
Audio files/Copd/192_2b2_Ar_mc_LittC2SE.wav going on
600 out of 792

(128, 372, 2)
Audio files/Copd/192_2b3_Al_mc_LittC2SE.wav going on
601 out of 792

(128, 372, 2)
Audio files/Copd/192_2b3_Ar_mc_LittC2SE.wav going on
602 out of 792

(128, 372, 2)
Audio files/Copd/193_1b2_Al_mc_AKGC417L.wav going on
603 out of 792

(128, 372, 2)
Audio files/Copd/193_1b2_Ar_mc_AKGC417L.wav going on
604 out of 792

(128, 372, 2)
Audio files/Copd/193_1b2_Ll_mc_AKGC417L.wav going on
605 out of 792

(128, 372, 2)
Audio files/Copd/193_1b2_Pl_mc_AKGC417L.wav going on
606 out of 792

(128, 372, 2)
Audio files/Copd/193_1b2_Pr_mc_AKGC417L.wav going on
607 out of 792

(128, 372, 2)
Audi

Audio files/Copd/205_1b3_Lr_mc_AKGC417L.wav going on
695 out of 792

(128, 372, 2)
Audio files/Copd/205_1b3_Pl_mc_AKGC417L.wav going on
696 out of 792

(128, 372, 2)
Audio files/Copd/205_1b3_Pr_mc_AKGC417L.wav going on
697 out of 792

(128, 372, 2)
Audio files/Copd/205_2b2_Pr_mc_AKGC417L.wav going on
698 out of 792

(128, 372, 2)
Audio files/Copd/205_2b3_Al_mc_AKGC417L.wav going on
699 out of 792

(128, 372, 2)
Audio files/Copd/205_2b3_Ar_mc_AKGC417L.wav going on
700 out of 792

(128, 372, 2)
Audio files/Copd/205_2b3_Ll_mc_AKGC417L.wav going on
701 out of 792

(128, 372, 2)
Audio files/Copd/205_2b4_Pl_mc_AKGC417L.wav going on
702 out of 792

(128, 372, 2)
Audio files/Copd/205_3b4_Al_mc_AKGC417L.wav going on
703 out of 792

(128, 372, 2)
Audio files/Copd/205_3b4_Ar_mc_AKGC417L.wav going on
704 out of 792

(128, 372, 2)
Audio files/Copd/205_3b4_Pl_mc_AKGC417L.wav going on
705 out of 792

(128, 372, 2)
Audio files/Copd/205_3b4_Pr_mc_AKGC417L.wav going on
706 out of 792

(128, 372, 2)
Audi

Audio files/Healthy/121_1b1_Tc_sc_Meditron.wav going on
2 out of 35

(128, 372, 2)
Audio files/Healthy/121_1p1_Tc_sc_Meditron.wav going on
3 out of 35

(128, 372, 2)
Audio files/Healthy/123_1b1_Al_sc_Meditron.wav going on
4 out of 35

(128, 372, 2)
Audio files/Healthy/125_1b1_Tc_sc_Meditron.wav going on
5 out of 35

(128, 372, 2)
Audio files/Healthy/126_1b1_Al_sc_Meditron.wav going on
6 out of 35

(128, 372, 2)
Audio files/Healthy/127_1b1_Ar_sc_Meditron.wav going on
7 out of 35

(128, 372, 2)
Audio files/Healthy/136_1b1_Ar_sc_Meditron.wav going on
8 out of 35

(128, 372, 2)
Audio files/Healthy/143_1b1_Al_sc_Meditron.wav going on
9 out of 35

(128, 372, 2)
Audio files/Healthy/144_1b1_Al_sc_Meditron.wav going on
10 out of 35

(128, 372, 2)
Audio files/Healthy/144_1b1_Tc_sc_Meditron.wav going on
11 out of 35

(128, 372, 2)
Audio files/Healthy/152_1b1_Al_sc_Meditron.wav going on
12 out of 35

(128, 372, 2)
Audio files/Healthy/153_1b1_Al_sc_Meditron.wav going on
13 out of 35

(128, 372, 2)


In [27]:
X7cut.shape

(95, 128, 372, 2)

In [60]:
X7.shape

(887, 128, 372, 2)

In [ ]:
np.save('X-STFTandMFCC-7-multiclips',X7)
np.save('y-STFTandMFCC-7-clips', y7)

In [12]:
np.save('X-STFTandMFCC-2000',X)

KeyboardInterrupt: 

In [9]:
np.save('y', y)

In [5]:
X=np.load('X.npy')

In [3]:
y=np.load('y.npy')